In [1]:
# Mexico
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 3
STATS_TYPES = ['Count','Percent','nshft'] #, 'pshft','growth', 'Labels']


In [2]:
# Data Notes for Brian
# Market Display Name for Mexico City needed a space 
# Add MapboxMarketName  - FIXED
# Transposed files still had BG rather than GEO  - FIXED
# San Luis Potosi - had blanks in MarketName and in file names
# Mexico City longitude wrong -99.13320799999997

In [3]:
#  Importing notes
# Use a map files from MX GEO 2020 Brian Stobie
# RFC 7946 warning: non-WGS84 GeoJSON output. Tip: use "-proj wgs84" to convert.
# https://mapshaper.org/
# open console in mapshaper and run the following command to output lat/long
# $ -proj wgs84

In [4]:
STATE_FILES = ["mexico_geo_geojson_2022_07_06.json"]
CENTER_STATE_FILES = ["geo_centers_ageb_2022_07_06.json"]

In [5]:

#  block_group_geojson_filename(state_name, census_year=2010)
# "geo_centers_ageb_2022_07_06.json"
def block_group_geojson_filename(state_name, census_year=2010):
    return STATE_FILES[0]

def bg_geojson_files_for_market(market_data, census_year):
    # states = market_data['states']
    # Dummy calculation for now
    states = ["mexico"]
    files = []
    for state in states:
        files.append(block_group_geojson_filename(state, census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "geo"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['GEO'])
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'MexicoCity'
# market_prefix= 'MexicoCity'
# MexicoCity__Mapping_GEO_Data.csv
# ./source_data/{market_file_prefix}__Mapping_GEO_Data.csv - the actual geo code data
# 
# MexicoCity__Mapping_Settings_Transposed File_GEO.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_GEO.csv - segment definitions for Geos


def file_type_for_data_type(data_type = "geo"):
    if data_type == "geo":
        return "GEO"
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# MexicoCity__Mapping_GEO_Data.csv
# ./source_data/{market_file_prefix}__Mapping_GEO_Data.csv - the actual geo data
# 

# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "geo"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}_Mapping_{file_type}_Data.csv'

# MexicoCity__Mapping_Settings_Transposed File_GEO.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_GEO.csv - segment definitions for GEOs

def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "geo"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "geo", published_data_base_path="/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{data_type}_data_v{MAP_VERSION}.json'

def published_map_settings_path(market_prefix,  data_type = "geo", published_data_base_path="/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    return f'{published_data_base_path}/{market_prefix.lower()}_{data_type}_settings_data_v{MAP_VERSION}.json'

def published_map_labels_path(market_prefix,  data_type = "geo", published_data_base_path="/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    return f'{published_data_base_path}/{market_prefix.lower()}_labels_v{MAP_VERSION}.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='geo'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows


def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    print('create_bg_features')
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['GEOID'])
        if (len(block_group) == 12) or (len(block_group) == 8):
            block_group = '0' + block_group
        # find the associated feature
        found = False
        if block_group in features:
            found = True
            feature = features[block_group]
            new_feature = copy.deepcopy(feature)
            properties = new_feature['properties']
            properties.pop("CVE_ENT",0)
            properties.pop("CVE_MUN",0)
            properties.pop("CVE_LOC",0)
            properties.pop("CVE_AGEB",0)
            new_feature['properties'] = properties
            for key in list(data_row.keys())[0:]:
                if key not in  ['GEOID','LAT','LON'] and stats_type in key: #and 'Count' in key:
                    if data_row[key] and data_row[key] != 'inf':
                        translated_key = map_settings_data[key]['mapbox_segment']
                        value = ast.literal_eval(data_row[key].replace(',',''))
                        value = int(value * 1000000) / 1000000
                        new_feature['properties'][translated_key] =  value
                if key == 'LAT':
                    new_feature['properties']['INTPTLAT10'] = data_row[key]
                if key == 'LON':
                    new_feature['properties']['INTPTLON10'] = data_row[key]
            new_features_to_keep.append(new_feature)
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)


def create_data_json(features):
    data = []
    for feature in features:
        #centroid = {
        #    "GEOID10": feature["properties"]["GEOID10"],
        #    "INTPTLAT10": feature["properties"]["INTPTLAT10"],
        #    "INTPTLON10": feature["properties"]["INTPTLON10"],
        #}
        datum = feature["properties"]
        data.append(datum)
    return data

def write_geojson(geojson_data, mapbox_market_file_prefix, data_type='geo', stats_type='Count',published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{mapbox_market_file_prefix.lower()}/data/{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}.json')
    
def write_data_json(json_data, mapbox_market_file_prefix, data_type='geo', stats_type='count',published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    json_string = json.dumps(json_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_data_v{MAP_VERSION}.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote centroid json data to  {published_data_base_path}/{mapbox_market_file_prefix.lower()}/data/{mapbox_market_file_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json')


def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "geo"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, mapbox_market_file_prefix, base_path="/Users/alanmccann/Dropbox/develop/bain-geospatial/public", data_type = "geo"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(mapbox_market_file_prefix.lower(),  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    

def create_bg_labels(market_file_prefix, mapbox_market_file_prefix, base_path="./source_data" , data_type = "geo" , published_data_base_path="/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix.lower(), base_path=base_path, data_type = "geo")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['GEOID']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LON']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(mapbox_market_file_prefix,  data_type , published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            row['CenterLat'] = float(row['CenterLat'])
            row['CenterLong'] = float(row['CenterLong'])
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/mexico_geospatial_settings_v{MAP_VERSION}.json', "w+")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/mexico_geospatial_settings_v{MAP_VERSION}.json')
    
    
def create_tileset_data_recipe_file(mapbox_market_file_prefix, published_data_base_path, stats_type):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{mapbox_market_file_prefix.lower()}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    print(recipe_file_path)

def create_tileset_labels_recipe_file(mapbox_market_file_prefix, published_data_base_path):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{mapbox_market_file_prefix.lower()}_labels_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{mapbox_market_file_prefix.lower()}_labels_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{mapbox_market_file_prefix.lower()}_labels_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    print(recipe_file_path)
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [6]:
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/mexico_map_source_data'
def create_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/mexico_map_source_data'):
    nationwide_features = {}
    for file in STATE_FILES:
        print(f'processing: {file}')
        geosjon_path = os.path.join(geojson_files_base_path,file)
        features = read_geo_gson_file(geosjon_path)
        for feature in features:
            key = feature['properties']["CVEGEO"]
            nationwide_features[key] = feature
    json_string = json.dumps(nationwide_features, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson data to  {geojson_files_base_path}/nationwide_bg.json')

def read_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/mexico_map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    # Opening JSON file
    f = open(geosjon_path)
    # returns JSON object as 
    # a dictionary
    features = json.load(f)
    # Closing file
    f.close()
    return features
# Different file for this?
def write_nation_bg_centroids_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/mexico_map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"geo_centers_ageb_2022_07_06.json")
    features = read_nation_bg_features_json(geojson_files_base_path)
    centroids = []
    for feature in features:
        centroid = {
            "CVEGEO": feature["properties"]["CVEGEO"],
            "INTPTLAT10": feature["geometry"]["coordinates"][1],
            "INTPTLON10": feature["properties"]["coordinates"][0],
        }
        centroids.append(centroid)
    json_string = json.dumps(centroids, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg_centroids.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote bg centroids data to  {geojson_files_base_path}/nationwide_bg_centroids.json')
    
        
        
    

In [7]:
def create_chart_json(published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json')



In [8]:
def create_comparison_chart_json(published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json')
    

In [9]:
def create_data_options_json(published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options_v{MAP_VERSION}.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options_v{MAP_VERSION}.json')


In [10]:
create_nation_bg_features_json()

processing: mexico_geo_geojson_2022_07_06.json
Loaded 81451 features from /Users/alanmccann/Dropbox/bain/mexico_map_source_data/mexico_geo_geojson_2022_07_06.json
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mexico_map_source_data/nationwide_bg.json


In [11]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    nationwide_bg_features = read_nation_bg_features_json(geojson_files_base_path)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows[:1]:
        #         if market_data['MarketName'] in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
        #             #if True:
        #             #print(market_data['MarketName'])
        if market_data['MarketName']: # in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName']
            mapbox_market_file_prefix =  market_data['MapboxMarketName']
            market_prefix = market_file_prefix

            for stats_type in STATS_TYPES:
                create_tileset_data_recipe_file(mapbox_market_file_prefix.lower(), published_data_base_path,stats_type)
                create_tileset_labels_recipe_file(mapbox_market_file_prefix.lower(), published_data_base_path)
            # Block Group Processing
            data_type = 'geo'
            map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "geo")
            map_settings = write_settings_data(map_settings_data, mapbox_market_file_prefix.lower(), base_path=published_data_base_path, data_type = "geo")
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            for stats_type in STATS_TYPES:
                features_to_use = create_bg_features(nationwide_bg_features, map_settings, block_group_rows, stats_type)

                new_geojson = create_geojson(features_to_use)
                write_geojson(new_geojson, mapbox_market_file_prefix, data_type, stats_type,published_data_base_path=published_data_base_path)
                if stats_type.lower() != 'labels':
                    new_data_json = create_data_json(features_to_use)
                    write_data_json(new_data_json, mapbox_market_file_prefix.lower(),data_type, stats_type.lower(),published_data_base_path)
            create_bg_labels(market_file_prefix, mapbox_market_file_prefix, base_path=source_data_base_path , data_type = "geo" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    create_geojson_settings(csv_file_path, published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public")
    print(f'*** COMPLETED {market_count} markets')
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
# published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010

In [12]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/mexico_map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/runJun14_6 Brian Stobie"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "/Users/alanmccann/Dropbox/develop/bain-geospatial/public"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
# Census year for geojson
census_year = 2010


# Process the data
process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
# create_chart_json(published_data_base_path)
# create_comparison_chart_json(published_data_base_path)
# create_data_options_json(published_data_base_path)


Market Data: 1
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_count_recipe_v3.json
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_labels_recipe_v3.json
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_percent_recipe_v3.json
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_labels_recipe_v3.json
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_nshft_recipe_v3.json
/Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_labels_recipe_v3.json
Loaded 138 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/runJun14_6 Brian Stobie/MexicoCity__Mapping_Settings_Transposed File_GEO.csv
Wrote settings data rows to /Users/alanmccann/Dropbox/bain/mtsds-data/mexicocity_geo_settings_data_v3.json
Loaded 5112 data rows from /Users/alanmccann/Dropbox/bain/bain-uploads/runJun14_6 Brian Stobie/MexicoCity_Mapping_GEO_Data.csv
create_bg_features
Processed 5112 features to use
Created GeoJson with  5112 features
Wrote geojson data to  /Users/alanmccann/Dropbox/bain/mtsds-data/mex

In [13]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MapboxMarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src_v{MAP_VERSION} {market_file_prefix}_bg_data_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION} --recipe {market_file_prefix}-bg-recipe_v{MAP_VERSION}.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION}')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels_v{MAP_VERSION}"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [14]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_v{MAP_VERSION}.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [15]:
def update_recipe(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}',
        f'{market_file_prefix.lower()}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def update_label_recipe(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix.lower()}_labels_v{MAP_VERSION}',
        f'{market_file_prefix.lower()}_labels_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', 
        f'{market_file_prefix.lower()}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix.lower()}_labels_v{MAP_VERSION}',
        '--recipe', f'{market_file_prefix.lower()}_labels_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix.lower()}_labels_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [16]:
import time
def publish_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix.lower()}_{stats_type.lower()}_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())


In [17]:
import time
def publish_label_tileset(market_file_prefix,working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix.lower()}_labels_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix.lower()}_labels_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())

In [19]:
import csv
import datetime
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/runJun14_6 Brian Stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
STATS_TYPES = ['count','percent','nshft'] #'growth'] #['count','percent','nshft','labels','pshft']
# STATS_TYPES = ['nshft']
for market_data in market_rows[1:]: #[339:]:
    #market_data['MarketName'] in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
    #market_file_prefix = market_data['MapboxMarketName'].lower()
    #if True: #market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
    if True: #market_data['MarketName'] in ['HiltonHeadIslandSC', 'HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
        market_file_prefix = market_data['MapboxMarketName'].lower()
        print('creating tilesets for:', market_file_prefix)
        print(datetime.datetime.now())
        for stats_type in STATS_TYPES:
            upload_source(market_file_prefix, stats_type, published_data_base_path)
            upload_tileset(market_file_prefix, stats_type, published_data_base_path)
            update_recipe(market_file_prefix, stats_type, published_data_base_path)
            publish_tileset(market_file_prefix, stats_type, published_data_base_path)
        upload_label_source(market_file_prefix, published_data_base_path)
        upload_label_tileset(market_file_prefix, published_data_base_path)
        update_label_recipe(market_file_prefix, published_data_base_path)
        publish_label_tileset(market_file_prefix, published_data_base_path)
        

creating tilesets for: monterrey
2023-06-15 11:56:09.529497
tilesets upload-source stobieb monterrey_count_src_v3 monterrey_count_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/monterrey_count_src_v3", "files": 1, "source_size": 5507202, "file_size": 5507202}', '']
tilesets create stobieb.monterrey_count_v3 --recipe monterrey_count_recipe_v3.json --name monterrey_count_v3
['{"message": "stobieb.monterrey_count_v3 already exists"}', ''] 
tilesets update-recipe stobieb.monterrey_count_v3 monterrey_count_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.monterrey_count_v3
2023-06-15 11:56:19.654529
['{"message": "Processing stobieb.monterrey_count_v3", "jobId": "clixbpltt000908mt1ynhggfc"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.monterrey_count_v3 or run tilesets job stobieb.monterrey_count_v3 clixbpltt000908mt1ynhggfc to view the status of your tileset.

{'message': 'Processing stobieb.monterrey_count_v3'

['{"message": "Processing stobieb.guadalajara_nshft_v3", "jobId": "clixcd6ap000a08lcg4lafa85"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.guadalajara_nshft_v3 or run tilesets job stobieb.guadalajara_nshft_v3 clixcd6ap000a08lcg4lafa85 to view the status of your tileset.

{'message': 'Processing stobieb.guadalajara_nshft_v3', 'jobId': 'clixcd6ap000a08lcg4lafa85'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 12:16:07.183679
tilesets upload-source stobieb guadalajara_labels_src_v3 guadalajara_labels_v3.json
['upload progress', '{"id": "mapbox://tileset-source/stobieb/guadalajara_labels_src_v3", "files": 2, "source_size": 559998, "file_size": 279864}', '']
tilesets create stobieb.guadalajara_labels_v3 --recipe guadalajara_labels_recipe_v3.json --name guadalajara_labels_v3
['{"message": "stobieb.guadalajara_labels_v3 already exists"}', ''] 
tilesets update-recipe stobieb.guada

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 12:33:27.582653
tilesets upload-source stobieb toluca_percent_src_v3 toluca_percent_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/toluca_percent_src_v3", "files": 1, "source_size": 3734777, "file_size": 3734777}', '']
tilesets create stobieb.toluca_percent_v3 --recipe toluca_percent_recipe_v3.json --name toluca_percent_v3
['{"message": "stobieb.toluca_percent_v3 already exists"}', ''] 
tilesets update-recipe stobieb.toluca_percent_v3 toluca_percent_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.toluca_percent_v3
2023-06-15 12:33:35.501177
['{"message": "Processing stobieb.toluca_percent_v3", "jobId": "clixd1ioh002208l0cd7kgzzs"}', ''] 
✔ Tileset job received. Visit https:

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 12:50:53.745134
creating tilesets for: leon
2023-06-15 12:50:53.745364
tilesets upload-source stobieb leon_count_src_v3 leon_count_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/leon_count_src_v3", "files": 1, "source_size": 2832773, "file_size": 2832773}', '']
tilesets create stobieb.leon_count_v3 --recipe leon_count_recipe_v3.json --name leon_count_v3
['{"message": "stobieb.leon_count_v3 already exists"}', ''] 
tilesets update-recipe stobieb.leon_count_v3 leon_count_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.leon_count_v3
2023-06-15 12:51:01.838862
['{"message": "Processing stobieb.leon_count_v3", "jobId": "clixdnyaf000a08mh746o1o0v"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.leon_count_v3 or run tilesets job stobieb.leon_count_v3 clixdnyaf000a08mh746o1o0v to view the statu

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 13:09:25.499007
tilesets upload-source stobieb queretaro_labels_src_v3 queretaro_labels_v3.json
['upload progress', '{"id": "mapbox://tileset-source/stobieb/queretaro_labels_src_v3", "files": 2, "source_size": 230994, "file_size": 115497}', '']
tilesets create stobieb.queretaro_labels_v3 --recipe queretaro_labels_recipe_v3.json --name queretaro_labels_v3
['{"message": "stobieb.queretaro_labels_v3 already exists"}', ''] 
tilesets update-recipe stobieb.queretaro_labels_v3 queretaro_labels_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.queretaro_labels_v3
2023-06-15 13:09:31.532058
['{"message": "Processing stobieb.queretaro_labels_v3", "jobId": "clixebqc4000d08me10dr3ues"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/sto

...

[''] Updated recipe.

tilesets publish stobieb.acapulco_labels_v3
2023-06-15 15:55:46.543613
['{"message": "Processing stobieb.acapulco_labels_v3", "jobId": "clixk9j8z005y08mo73uma7sw"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.acapulco_labels_v3 or run tilesets job stobieb.acapulco_labels_v3 clixk9j8z005y08mo73uma7sw to view the status of your tileset.

{'message': 'Processing stobieb.acapulco_labels_v3', 'jobId': 'clixk9j8z005y08mo73uma7sw'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 15:59:07.996187
creating tilesets for: tuxtlagutierrez
2023-06-15 15:59:07.996357
tilesets upload-source stobieb tuxtlagutierrez_count_src_v3 tuxtlagutierrez_count_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb

[''] Updated recipe.

tilesets publish stobieb.matamoros_percent_v3
2023-06-15 16:12:27.543464
['{"message": "Processing stobieb.matamoros_percent_v3", "jobId": "clixkuzvw000c08l88stjbzzf"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.matamoros_percent_v3 or run tilesets job stobieb.matamoros_percent_v3 clixkuzvw000c08l88stjbzzf to view the status of your tileset.

{'message': 'Processing stobieb.matamoros_percent_v3', 'jobId': 'clixkuzvw000c08l88stjbzzf'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 16:15:50.480647
tilesets upload-source stobieb matamoros_nshft_src_v3 matamoros_nshft_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/matamoros_nshft_src_v3", "files": 1, "source_size": 685776, "file_s

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 16:31:58.417363
creating tilesets for: nuevalaredo
2023-06-15 16:31:58.417572
tilesets upload-source stobieb nuevalaredo_count_src_v3 nuevalaredo_count_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/nuevalaredo_count_src_v3", "files": 1, "source_size": 608244, "file_size": 608244}', '']
tilesets create stobieb.nuevalaredo_count_v3 --recipe nuevalaredo_count_recipe_v3.json --name nuevalaredo_count_v3
['{"message": "Successfully created empty tileset stobieb.nuevalaredo_count_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.nuevalaredo_count_v3 nuevalaredo_count_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.nuevalaredo_count_v3
2023-06-15 16:32:05.387613
['{"message": "Processing stobieb.nuevalaredo_count_v3", "jobId": "clixlk8lh004608l42z1k3as4"}', ''] 

['upload progress', '{"id": "mapbox://tileset-source/stobieb/nogales_nshft_src_v3", "files": 1, "source_size": 474118, "file_size": 474118}', '']
tilesets create stobieb.nogales_nshft_v3 --recipe nogales_nshft_recipe_v3.json --name nogales_nshft_v3
['{"message": "Successfully created empty tileset stobieb.nogales_nshft_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.nogales_nshft_v3 nogales_nshft_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.nogales_nshft_v3
2023-06-15 16:47:51.441905
['{"message": "Processing stobieb.nogales_nshft_v3", "jobId": "clixm4ihq001v08l656ic053t"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.nogales_nshft_v3 or run tilesets job stobieb.nogales_nshft_v3 clixm4ihq001v08l656ic053t to view the status of your tileset.

{'message': 'Processing stobieb.nogales_nshft_v3', 'jobId': 'clixm4ihq001v08l656ic053t'}
processing: sleeping
processing: sleeping
p

[''] Updated recipe.

tilesets publish stobieb.monclova_labels_v3
2023-06-15 17:04:32.392793
['{"message": "Processing stobieb.monclova_labels_v3", "jobId": "clixmpytv003e08l6142w5od9"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.monclova_labels_v3 or run tilesets job stobieb.monclova_labels_v3 clixmpytv003e08l6142w5od9 to view the status of your tileset.

{'message': 'Processing stobieb.monclova_labels_v3', 'jobId': 'clixmpytv003e08l6142w5od9'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-06-15 17:07:57.678165
creating tilesets for: mexico
2023-06-15 17:07:57.678315
tilesets upload-source stobieb mexico_count_src_v3 mexico_count_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/mexico_count_src_v3", "fil